In [ ]:
import json
import requests
import urllib.parse

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## xAPIステートメントを取得

<span style="color:red"><b>Learning Lockerで作成したLRSのクライアント情報を設定すること。</b></span>

In [ ]:
user = '<Key>'
passwd = '<Secret>'

[AGGREGATION HTTP INTERFACE](https://learninglocker.atlassian.net/wiki/spaces/DOCS/pages/106037259/Aggregation+API)で検索条件( `$match` )と件数( `$limit` )を指定し、ステートメントを取得する。  
ステートメントの詳細は[xAPI仕様](https://github.com/adlnet/xAPI-Spec)([日本語版](https://github.com/elc-gh/xAPI-Spec_ja))を参照すること。

In [ ]:
pipeline = urllib.parse.quote(json.dumps([
    {'$project': {
        'timestamp': 1,
        'statement': 1,
        '_id': 0
    }},
    # 検索条件
    {'$match': {
        #'statement.actor.name': '...', # ユーザの絞り込み
        #'statement.verb.display.en': '...', # 操作の絞り込み
        'timestamp': { # 操作日時の絞り込み
            "$gte": {
                '$dte': '2022-04-01T00:00:00Z'
            },
            "$lte": {
                '$dte': '9999-12-31T00:00:00Z'
            }
        }
    }},
    # 取得する最大件数
    {'$limit': 30000},
]))
url = f'http://learninglocker:8080/statements/aggregate?cache=false&pipeline={pipeline}'

r = requests.get(url, auth=(user, passwd))

# Jsonを辞書型に変換
response_data = json.loads(r.text)
# 辞書からステートメントのみを抽出
stats = [d['statement'] for d in response_data]
# ネスト項目をフラットにしてpandasのDataFrameを作成
df = pd.io.json.json_normalize(stats)
pd.set_option('display.max_columns', 100)
df.head()

In [ ]:
# イベントの件数を集計
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 20
sns.countplot(y='verb.display.en', data=df)